# ETL Project - Transform

In [110]:
import pandas as pd
import numpy as np

## Import the following files for Transformation

###  Netflix
- netflix_final.csv
- movies_revenue.csv

### Budget
- movies_num_final.csv
- movies_budget_df.csv
- movies_budget_df_revenue.csv

## Step I:  Merge

### Netflix
- Merge netflix_final and movies_revenue on title: **netflix_revenu_df**


### Budget
- Merge movies_budget_df and movies_budget_df_revenue on id: **movies_budget_revenue_df**

## Step II:  Clean and Wrangle


###  **netflix_revenu_df**

- Inspect dataframe
- Check for missing values
- Drop Nan values and not zero values !IMPORTANT
- Convert numeric columns to numeric if necessary
- Save to a new dataframe and export as csv

###  **movies_budget_revenue_df**

- Inspect dataframe
- Check for missing values
- Drop Nan values and not zero values !IMPORTANT
- Remove $ and (,) in numeric columns
- Convert numeric columns to numeric if necessary
- Save to a new dataframe and export as csv

## Step 1

### Netflix Revenue Merge

In [104]:
# Import netflix_final.csv
netflix_final = pd.read_csv('netflix_final.csv')
netflix_final.tail()

,title,country,release_year,rating
4260,Frank and Cindy,United States,2007,TV-MA
4261,Frank and Cindy,United States,2015,R
4262,Iverson,United States,2014,NR
4263,Jeremy Scott: The People's Designer,United States,2015,PG-13
4264,Little Baby Bum: Nursery Rhyme Friends,NaN,2016,NaN


In [105]:
# Import movie_revenue.csv
movie_revenue = pd.read_csv('movie_revenue.csv')
movie_revenue.head()

,id,title,revenue,vote_count,vote_average,runtime,popularity
0,601131,Norm of the North: King Sized Adventure,1442504,4,7.6,90.0,8.420
1,415722,Jandino: Whatever it Takes,0,2,5.0,94.0,0.600
2,262543,Automata,0,1171,5.8,110.0,25.466
3,474590,Fabrizio Copano: solo pienso en mí,0,4,5.3,59.0,1.126
4,262338,Good People,0,324,5.5,90.0,28.195


In [106]:
# Merge netflix_final and movie_revenue on title
netflix_revenue_df = pd.merge(netflix_final, movie_revenue, on = 'title')
netflix_revenue_df.head()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019,TV-PG,601131,1442504,4,7.6,90.0,8.420
1,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,415722,0,2,5.0,94.0,0.600
2,Automata,"Bulgaria, United States, Spain, Canada",2014,R,262543,0,1171,5.8,110.0,25.466
3,Good People,"United States, United Kingdom, Denmark, Sweden",2014,R,262338,0,324,5.5,90.0,28.195
4,Joaquín Reyes: Una y no más,NaN,2017,TV-MA,474599,0,1,6.0,77.0,0.600


In [7]:
# Inspect values and data types
netflix_revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3217 entries, 0 to 3216
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3217 non-null   object 
 1   country       3107 non-null   object 
 2   release_year  3217 non-null   int64  
 3   rating        3215 non-null   object 
 4   id            3217 non-null   int64  
 5   revenue       3217 non-null   int64  
 6   vote_count    3217 non-null   int64  
 7   vote_average  3217 non-null   float64
 8   runtime       3188 non-null   float64
 9   popularity    3217 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 276.5+ KB


In [8]:
# check for na values in columns that have fewer columns (less than 3217)

# Country Column
# Check for number of unique values
country_unique = netflix_revenue_df['country'].nunique()
print(f'There are {country_unique} groups of countries in the dataset')

country_null_values_sum = netflix_revenue_df['country'].isna().sum()
print(f'There are {country_null_values_sum} NaN values in the dataset')

There are 404 groups of countries in the dataset
There are 110 NaN values in the dataset


In [107]:
# Extract and inspect the NaN values
country_null_values = netflix_revenue_df.loc[netflix_revenue_df['country'].isna()]
country_null_values = country_null_values.reset_index(drop = True)

In [10]:
# Eliminating columns with these NaN values will impact the list. We will extract the country names 
# by using titles and is to perform
# API call to the TMDB data base

In [111]:
# Execute API call to TMDB
from config import tmdb_api_key
import requests 
import locale 
locale.setlocale( locale.LC_ALL, '' )

'English_United States.1252'

In [108]:
country_null_values.head()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Joaquín Reyes: Una y no más,NaN,2017,TV-MA,474599,0,1,6.0,77.0,0.600
1,Out of Thin Air,NaN,2017,TV-14,455479,0,35,6.3,85.0,3.476
2,Cultivating the Seas: History and Future of th...,NaN,2019,TV-G,648825,0,0,0.0,45.0,0.634
3,Jugaad,NaN,2017,TV-14,15646,0,3,5.3,114.0,0.751
4,Evelyn,NaN,2019,TV-MA,21868,0,61,6.5,92.0,8.942


In [113]:
# API call to extract country values
count = 0
countries = []


for index, row in country_null_values.iterrows():
    
    count = count + 1
    print(f'{count} | Extracting country information for Movie-ID: {row["id"]}')
    print('........................')
    
    # Execute API call with TMDB API endpoint
    row['response'] = requests.get(f'https://api.themoviedb.org/3/movie/{row["id"]}?api_key={tmdb_api_key}').json()
    
    # Extract production_countries
    countries_response = row['response']['production_countries']
    
    # Append to countries list
    countries.append(countries_response)    

1 | Extracting country information for Movie-ID: 474599
........................
2 | Extracting country information for Movie-ID: 455479
........................
3 | Extracting country information for Movie-ID: 648825
........................
4 | Extracting country information for Movie-ID: 15646
........................
5 | Extracting country information for Movie-ID: 21868
........................
6 | Extracting country information for Movie-ID: 563801
........................
7 | Extracting country information for Movie-ID: 552752
........................
8 | Extracting country information for Movie-ID: 462469
........................
9 | Extracting country information for Movie-ID: 550248
........................
10 | Extracting country information for Movie-ID: 528530
........................
11 | Extracting country information for Movie-ID: 641816
........................
12 | Extracting country information for Movie-ID: 554175
........................
13 | Extracting country inf

102 | Extracting country information for Movie-ID: 609059
........................
103 | Extracting country information for Movie-ID: 468366
........................
104 | Extracting country information for Movie-ID: 597567
........................
105 | Extracting country information for Movie-ID: 651802
........................
106 | Extracting country information for Movie-ID: 589871
........................
107 | Extracting country information for Movie-ID: 596204
........................
108 | Extracting country information for Movie-ID: 300821
........................
109 | Extracting country information for Movie-ID: 595503
........................
110 | Extracting country information for Movie-ID: 500177
........................


In [114]:
# Inspect countries
countries

[[],
 [{'iso_3166_1': 'IS', 'name': 'Iceland'},
  {'iso_3166_1': 'GB', 'name': 'United Kingdom'}],
 [{'iso_3166_1': 'JP', 'name': 'Japan'}],
 [{'iso_3166_1': 'IN', 'name': 'India'}],
 [{'iso_3166_1': 'IE', 'name': 'Ireland'},
  {'iso_3166_1': 'US', 'name': 'United States of America'}],
 [{'iso_3166_1': 'US', 'name': 'United States of America'}],
 [{'iso_3166_1': 'US', 'name': 'United States of America'}],
 [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}],
 [{'iso_3166_1': 'US', 'name': 'United States of America'}],
 [],
 [{'iso_3166_1': 'IN', 'name': 'India'}],
 [{'iso_3166_1': 'US', 'name': 'United States of America'}],
 [],
 [],
 [],
 [],
 [],
 [{'iso_3166_1': 'IN', 'name': 'India'}],
 [{'iso_3166_1': 'CA', 'name': 'Canada'},
  {'iso_3166_1': 'US', 'name': 'United States of America'}],
 [{'iso_3166_1': 'BR', 'name': 'Brazil'}],
 [{'iso_3166_1': 'UY', 'name': 'Uruguay'}],
 [],
 [{'iso_3166_1': 'IN', 'name': 'India'}],
 [{'iso_3166_1': 'CA', 'name': 'Canada'}],
 [{'iso_3166_1': 'ES', '

In [135]:
# Extract the first item in the countries list
import numpy as np
country_complete = []

for x in countries:
    
    
    if x:
        
        # Append first item if list is not empty
        country_complete.append(x[0]['name'])
    else:
        # Assign null values if x is null
        x = np.NaN
        country_complete.append(x)
country_complete

[nan,
 'Iceland',
 'Japan',
 'India',
 'Ireland',
 'United States of America',
 'United States of America',
 'United Kingdom',
 'United States of America',
 nan,
 'India',
 'United States of America',
 nan,
 nan,
 nan,
 nan,
 nan,
 'India',
 'Canada',
 'Brazil',
 'Uruguay',
 nan,
 'India',
 'Canada',
 'Spain',
 nan,
 'United States of America',
 'United States of America',
 nan,
 nan,
 'United States of America',
 nan,
 'Canada',
 nan,
 'United States of America',
 'United States of America',
 'Mexico',
 'United States of America',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'United Kingdom',
 nan,
 'South Korea',
 'United States of America',
 'India',
 'Egypt',
 'United States of America',
 'Egypt',
 nan,
 'United States of America',
 'United States of America',
 'India',
 'Indonesia',
 'United States of America',
 'United States of America',
 'United States of America',
 nan,
 'United States of America',
 'Chile',
 'United States of America',
 'Brazil',
 'United States of America',
 '

In [136]:
# Drop the country column with nan values
country_null_values.drop('country', axis = 1)

,title,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Joaquín Reyes: Una y no más,2017,TV-MA,474599,0,1,6.0,77.0,0.600
1,Out of Thin Air,2017,TV-14,455479,0,35,6.3,85.0,3.476
2,Cultivating the Seas: History and Future of th...,2019,TV-G,648825,0,0,0.0,45.0,0.634
3,Jugaad,2017,TV-14,15646,0,3,5.3,114.0,0.751
4,Evelyn,2019,TV-MA,21868,0,61,6.5,92.0,8.942
...,...,...,...,...,...,...,...,...,...
105,Grass Is Greener,2019,TV-MA,589871,0,23,6.9,97.0,6.734
106,Brené Brown: The Call to Courage,2019,TV-14,596204,0,20,7.3,76.0,4.715
107,My Wife and My Wifey,2014,TV-14,300821,0,3,3.7,0.0,0.600
108,Super Monsters Furever Friends,2019,TV-Y,595503,0,7,5.6,58.0,26.843


In [137]:
# Insert a new country column with values from the API call
country_null_values['country'] = country_complete

In [138]:
# Inspect the dataframe
country_null_values

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Joaquín Reyes: Una y no más,NaN,2017,TV-MA,474599,0,1,6.0,77.0,0.600
1,Out of Thin Air,Iceland,2017,TV-14,455479,0,35,6.3,85.0,3.476
2,Cultivating the Seas: History and Future of th...,Japan,2019,TV-G,648825,0,0,0.0,45.0,0.634
3,Jugaad,India,2017,TV-14,15646,0,3,5.3,114.0,0.751
4,Evelyn,Ireland,2019,TV-MA,21868,0,61,6.5,92.0,8.942
...,...,...,...,...,...,...,...,...,...,...
105,Grass Is Greener,NaN,2019,TV-MA,589871,0,23,6.9,97.0,6.734
106,Brené Brown: The Call to Courage,NaN,2019,TV-14,596204,0,20,7.3,76.0,4.715
107,My Wife and My Wifey,Egypt,2014,TV-14,300821,0,3,3.7,0.0,0.600
108,Super Monsters Furever Friends,NaN,2019,TV-Y,595503,0,7,5.6,58.0,26.843


In [139]:
# Merge country_null_values and netflix_revenue_df on title
netflix_revenue_merged = netflix_revenue_df.merge(country_null_values, on = 'title', how = 'left')

In [140]:
netflix_revenue_merged.tail()

,title,country_x,release_year_x,rating_x,id_x,revenue_x,vote_count_x,vote_average_x,runtime_x,popularity_x,country_y,release_year_y,rating_y,id_y,revenue_y,vote_count_y,vote_average_y,runtime_y,popularity_y
3214,Frank and Cindy,United States,2007,TV-MA,345170,0,13,5.5,102.0,6.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3215,Frank and Cindy,United States,2015,R,345170,0,13,5.5,102.0,6.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3216,Frank and Cindy,United States,2015,R,345170,0,13,5.5,102.0,6.039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,Iverson,United States,2014,NR,266442,0,67,7.2,87.0,8.577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,Jeremy Scott: The People's Designer,United States,2015,PG-13,360354,0,12,7.6,110.0,2.461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
# drop extra columns after merge
netflix_revenue_dropped = netflix_revenue_merged.drop(['country_y', 'release_year_y', 'rating_y', 'id_y', 'revenue_y', 'vote_count_y', 'vote_average_y', 'runtime_y', 'popularity_y'], axis = 1)

In [142]:
netflix_revenue_renamed = netflix_revenue_dropped.rename(columns={'country_x': 'country', 'release_year_x': 'release_year', 'rating_x': 'rating', 'id_x': 'id', 'revenue_x': 'revenue', 'vote_count_x': 'vote_count', 'vote_average_x': 'vote_average', 'runtime_x': 'runtime', 'popularity_x': 'popularity'})

In [143]:
netflix_revenue_renamed.tail()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
3214,Frank and Cindy,United States,2007,TV-MA,345170,0,13,5.5,102.0,6.039
3215,Frank and Cindy,United States,2015,R,345170,0,13,5.5,102.0,6.039
3216,Frank and Cindy,United States,2015,R,345170,0,13,5.5,102.0,6.039
3217,Iverson,United States,2014,NR,266442,0,67,7.2,87.0,8.577
3218,Jeremy Scott: The People's Designer,United States,2015,PG-13,360354,0,12,7.6,110.0,2.461


In [144]:
# Inspect rating
ratings_null = netflix_revenue_renamed.loc[netflix_revenue_renamed['rating'].isna()]

In [145]:
ratings_null

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
104,Louis C.K.: Hilarious,United States,2010,NaN,45523,0,167,7.9,84.0,7.369
1402,My Honor Was Loyalty,Italy,2015,NaN,383001,0,20,5.2,116.0,3.526


In [146]:
# Simple google search reveals both NaN values are of TV-MA ratings and the values are replaced

In [147]:
netflix_revenue_renamed['rating'] = netflix_revenue_renamed['rating'].fillna('TV-MA')

In [148]:
netflix_revenue_renamed.head()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019,TV-PG,601131,1442504,4,7.6,90.0,8.420
1,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,415722,0,2,5.0,94.0,0.600
2,Automata,"Bulgaria, United States, Spain, Canada",2014,R,262543,0,1171,5.8,110.0,25.466
3,Good People,"United States, United Kingdom, Denmark, Sweden",2014,R,262338,0,324,5.5,90.0,28.195
4,Joaquín Reyes: Una y no más,NaN,2017,TV-MA,474599,0,1,6.0,77.0,0.600


In [149]:
# Inspect runtime
runtime_null = netflix_revenue_renamed['runtime'].isna().sum()
print(runtime_null)

# To avoid dropping that many rows, we fill the nan values with average runtime of movies which is 97 minutes according to
# https://stephenfollows.com/netflix-original-movies-shows

29


In [150]:
# replace the NaNs in runtime with 97
netflix_revenue_renamed['runtime'] = netflix_revenue_renamed['runtime'].fillna(97)


In [151]:
netflix_revenue_renamed['runtime'].isna().sum()

0

In [152]:
# Inspect the full dataframe
netflix_revenue_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3219 entries, 0 to 3218
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3219 non-null   object 
 1   country       3107 non-null   object 
 2   release_year  3219 non-null   int64  
 3   rating        3219 non-null   object 
 4   id            3219 non-null   int64  
 5   revenue       3219 non-null   int64  
 6   vote_count    3219 non-null   int64  
 7   vote_average  3219 non-null   float64
 8   runtime       3219 non-null   float64
 9   popularity    3219 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 276.6+ KB


In [153]:
# Check for duplicate rows
netflix_revenue_renamed_duplicated = netflix_revenue_renamed.duplicated()

In [154]:
netflix_revenue_renamed_duplicated

0       False
1       False
2       False
3       False
4       False
        ...  
3214     True
3215    False
3216     True
3217    False
3218    False
Length: 3219, dtype: bool

In [155]:
# Drop duplicate rows
netflix_revenue_renamed = netflix_revenue_renamed.drop_duplicates()

In [156]:
netflix_revenue_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3148 entries, 0 to 3218
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3148 non-null   object 
 1   country       3039 non-null   object 
 2   release_year  3148 non-null   int64  
 3   rating        3148 non-null   object 
 4   id            3148 non-null   int64  
 5   revenue       3148 non-null   int64  
 6   vote_count    3148 non-null   int64  
 7   vote_average  3148 non-null   float64
 8   runtime       3148 non-null   float64
 9   popularity    3148 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 270.5+ KB


In [157]:
# Replace the NaN coutry values with 'unknown'
netflix_revenue_renamed['country'] = netflix_revenue_renamed['country'].fillna('unknown')

In [158]:
netflix_revenue_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3148 entries, 0 to 3218
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3148 non-null   object 
 1   country       3148 non-null   object 
 2   release_year  3148 non-null   int64  
 3   rating        3148 non-null   object 
 4   id            3148 non-null   int64  
 5   revenue       3148 non-null   int64  
 6   vote_count    3148 non-null   int64  
 7   vote_average  3148 non-null   float64
 8   runtime       3148 non-null   float64
 9   popularity    3148 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 270.5+ KB


In [100]:
# Select first country in the country column to have one observation per row


In [159]:
netflix_revenue_renamed.head()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",2019,TV-PG,601131,1442504,4,7.6,90.0,8.420
1,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,415722,0,2,5.0,94.0,0.600
2,Automata,"Bulgaria, United States, Spain, Canada",2014,R,262543,0,1171,5.8,110.0,25.466
3,Good People,"United States, United Kingdom, Denmark, Sweden",2014,R,262338,0,324,5.5,90.0,28.195
4,Joaquín Reyes: Una y no más,unknown,2017,TV-MA,474599,0,1,6.0,77.0,0.600


In [161]:
# Strip the country column values and select the first country
stripped_country = [x.split(',') for x in netflix_revenue_renamed['country']]

In [163]:
netflix_revenue_renamed['country'] = [x[0] for x in stripped_country]

In [165]:
netflix_revenue_renamed.tail()

,title,country,release_year,rating,id,revenue,vote_count,vote_average,runtime,popularity
3212,Toro,Spain,2016,NR,364615,1903083,115,6.0,105.0,6.963
3213,Frank and Cindy,United States,2007,TV-MA,345170,0,13,5.5,102.0,6.039
3215,Frank and Cindy,United States,2015,R,345170,0,13,5.5,102.0,6.039
3217,Iverson,United States,2014,NR,266442,0,67,7.2,87.0,8.577
3218,Jeremy Scott: The People's Designer,United States,2015,PG-13,360354,0,12,7.6,110.0,2.461


In [167]:
netflix_revenue_renamed = netflix_revenue_renamed.reset_index(drop = True)

In [169]:
netflix_revenue_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3148 entries, 0 to 3147
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3148 non-null   object 
 1   country       3148 non-null   object 
 2   release_year  3148 non-null   int64  
 3   rating        3148 non-null   object 
 4   id            3148 non-null   int64  
 5   revenue       3148 non-null   int64  
 6   vote_count    3148 non-null   int64  
 7   vote_average  3148 non-null   float64
 8   runtime       3148 non-null   float64
 9   popularity    3148 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 246.1+ KB


In [170]:
# Rearrange dataframe to have id as first column
netflix_revenue_final = netflix_revenue_renamed[['id', 'title', 'country', 'release_year', 'rating', 'revenue', 'vote_count',\
                                                'vote_average', 'runtime', 'popularity']]

In [171]:
netflix_revenue_final.head()

,id,title,country,release_year,rating,revenue,vote_count,vote_average,runtime,popularity
0,601131,Norm of the North: King Sized Adventure,United States,2019,TV-PG,1442504,4,7.6,90.0,8.420
1,415722,Jandino: Whatever it Takes,United Kingdom,2016,TV-MA,0,2,5.0,94.0,0.600
2,262543,Automata,Bulgaria,2014,R,0,1171,5.8,110.0,25.466
3,262338,Good People,United States,2014,R,0,324,5.5,90.0,28.195
4,474599,Joaquín Reyes: Una y no más,unknown,2017,TV-MA,0,1,6.0,77.0,0.600


In [172]:
# Export netflix_revenue_final to csv file
netflix_revenue_final.to_csv('netflix_movies_revenue.csv', index = False)

### Movies Budget Merge

In [3]:
# Import movies_budget_df and movies_budget_df_revenue
movies_budget_df = pd.read_csv('movies_budget_df.csv')
movies_budget_df.head()

,id,title,vote_count,vote_average
0,299534,Avengers: Endgame,15589,8.3
1,1865,Pirates of the Caribbean: On Stranger Tides,10375,6.5
2,99861,Avengers: Age of Ultron,16755,7.3
3,299536,Avengers: Infinity War,20088,8.3
4,285,Pirates of the Caribbean: At World's End,10329,7.2


In [4]:
movies_budget_df_revenue = pd.read_csv('movies_budget_df_revenue.csv')
movies_budget_df_revenue.head()

,id,revenue,runtime,popularity
0,299534,2797800564,181.0,193.659
1,1865,1045713802,136.0,79.193
2,99861,1405403694,141.0,85.183
3,299536,2046239637,149.0,229.850
4,285,961000000,169.0,77.611


In [5]:
# Merge movies_budget_df_revenue and movies_budget_df on id
movies_budget_revenue_merged = pd.merge(movies_budget_df, movies_budget_df_revenue, on = 'id')

In [6]:
movies_budget_revenue_merged.head()

,id,title,vote_count,vote_average,revenue,runtime,popularity
0,299534,Avengers: Endgame,15589,8.3,2797800564,181.0,193.659
1,1865,Pirates of the Caribbean: On Stranger Tides,10375,6.5,1045713802,136.0,79.193
2,99861,Avengers: Age of Ultron,16755,7.3,1405403694,141.0,85.183
3,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850
4,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850


In [7]:
# Inspect the merged dataframe
movies_budget_revenue_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 0 to 6480
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            6481 non-null   int64  
 1   title         6481 non-null   object 
 2   vote_count    6481 non-null   int64  
 3   vote_average  6481 non-null   float64
 4   revenue       6481 non-null   int64  
 5   runtime       6474 non-null   float64
 6   popularity    6481 non-null   float64
dtypes: float64(3), int64(3), object(1)
memory usage: 405.1+ KB


In [8]:
# check for duplicates
movies_budget_revenue_merged_duplicated = movies_budget_revenue_merged[movies_budget_revenue_merged.duplicated()]
movies_budget_revenue_merged_duplicated

,id,title,vote_count,vote_average,revenue,runtime,popularity
4,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850
5,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850
6,299536,Avengers: Infinity War,20088,8.3,2046239637,149.0,229.850
15,420818,The Lion King,7008,7.2,1656943394,118.0,182.649
16,420818,The Lion King,7008,7.2,1656943394,118.0,182.649
...,...,...,...,...,...,...,...
5806,401246,The Square,867,6.8,1502347,151.0,14.416
5807,401246,The Square,867,6.8,1502347,151.0,14.416
6123,84184,Celeste & Jesse Forever,298,6.5,3094813,91.0,11.090
6124,84184,Celeste & Jesse Forever,298,6.5,3094813,91.0,11.090


In [12]:
# Drop duplicate
movies_budget_revenue_merged = movies_budget_revenue_merged.drop_duplicates(keep = 'last')

In [13]:
movies_budget_revenue_merged = movies_budget_revenue_merged.reset_index(drop = True)

In [14]:
movies_budget_revenue_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5686 entries, 0 to 5685
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            5686 non-null   int64  
 1   title         5686 non-null   object 
 2   vote_count    5686 non-null   int64  
 3   vote_average  5686 non-null   float64
 4   revenue       5686 non-null   int64  
 5   runtime       5679 non-null   float64
 6   popularity    5686 non-null   float64
dtypes: float64(3), int64(3), object(1)
memory usage: 311.1+ KB


In [15]:
# Export movies_budget_revenue_merged to csv file: all_budget_revenue_final
movies_budget_revenue_merged.to_csv('all_budget_revenue_final.csv', index = False)

In [77]:
# Import movies_num_final.csv
movies_num_final = pd.read_csv('movies_num_final.csv')
movies_num_final.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,396,099,202"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,065,478,084"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"


In [78]:
# Inspect movies_num_final
movies_num_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6000 non-null   object
 1   Movie             6000 non-null   object
 2   ProductionBudget  6000 non-null   object
 3   DomesticGross     6000 non-null   object
 4   WorldwideGross    6000 non-null   object
dtypes: object(5)
memory usage: 234.5+ KB


In [79]:
# Check for duplicates
movies_num_final_duplicated = movies_num_final[movies_num_final.duplicated()].sum()

In [80]:
movies_num_final_duplicated

ReleaseDate         0.0
Movie               0.0
ProductionBudget    0.0
DomesticGross       0.0
WorldwideGross      0.0
dtype: float64

In [81]:
# Remove currency ($) and commas(,) in ProductionBudget, DomesticGross and WorldwideGross
# Create a list of the three columns
budget_columns = [movies_num_final['ProductionBudget'], movies_num_final['DomesticGross'], movies_num_final['WorldwideGross']]
BudgetFormatted = []
for x in budget_columns:
    
    budget_formatted = x.str.replace('$', '').str.replace(',', '').astype('int64')
    BudgetFormatted.append(budget_formatted)
    

In [82]:
# Unpack list to respective columns
movies_num_final['ProductionBudget'], movies_num_final['DomesticGross'], movies_num_final['WorldwideGross'] = [x for x in BudgetFormatted]

In [83]:
movies_num_final.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,400000000,858373000,2797800564
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,379000000,241063875,1045663875
2,"Apr 22, 2015",Avengers: Age of Ultron,365000000,459005868,1396099202
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2065478084
4,"Apr 25, 2018",Avengers: Infinity War,300000000,678815482,2044540523


In [58]:
movies_num_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6000 non-null   object
 1   Movie             6000 non-null   object
 2   ProductionBudget  6000 non-null   int64 
 3   DomesticGross     6000 non-null   int64 
 4   WorldwideGross    6000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 234.5+ KB


In [84]:
import datetime
from dateutil.parser import parse

In [85]:
# Convert ReleaseDate to datetime object
movies_num_final['ReleaseDate'] = pd.to_datetime(movies_num_final['ReleaseDate'], errors ='coerce')

In [88]:
movies_num_final.tail()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
5995,NaT,Hayride,60000,0,0
5996,1962-08-10,The Brain That Wouldn't Die,60000,0,0
5997,2013-10-04,The Dirties,55000,0,0
5998,1995-08-09,The Brothers McMullen,50000,10426506,10426506
5999,2001-03-16,Gabriela,50000,2335352,2335352


In [92]:
# Drop null values
movies_drop_nulltime = movies_num_final[pd.notnull(movies_num_final['ReleaseDate'])]

In [94]:
movies_drop_nulltime.tail()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
5994,2015-08-28,Turbo Kid,60000,0,0
5996,1962-08-10,The Brain That Wouldn't Die,60000,0,0
5997,2013-10-04,The Dirties,55000,0,0
5998,1995-08-09,The Brothers McMullen,50000,10426506,10426506
5999,2001-03-16,Gabriela,50000,2335352,2335352


In [95]:
# Reset index
movies_drop_nulltime = movies_drop_nulltime.reset_index(drop = True)

In [97]:
movies_drop_nulltime.tail()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
5887,2015-08-28,Turbo Kid,60000,0,0
5888,1962-08-10,The Brain That Wouldn't Die,60000,0,0
5889,2013-10-04,The Dirties,55000,0,0
5890,1995-08-09,The Brothers McMullen,50000,10426506,10426506
5891,2001-03-16,Gabriela,50000,2335352,2335352


In [98]:
movies_drop_nulltime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5892 entries, 0 to 5891
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ReleaseDate       5892 non-null   datetime64[ns]
 1   Movie             5892 non-null   object        
 2   ProductionBudget  5892 non-null   int64         
 3   DomesticGross     5892 non-null   int64         
 4   WorldwideGross    5892 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 230.3+ KB


In [99]:
# Export movies_drop_nulltime to csv as MoviesFinalBudget
movies_drop_nulltime.to_csv('MoviesFinalBudget.csv', index = False)